In [ ]:
!pip install -qU langchain langchain-fireworks requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain_fireworks import ChatFireworks
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory
import getpass
import os

In [ ]:
os.environ["FIREWORKS_API_KEY"] = getpass("🚀 Enter your Fireworks API key: ")

🚀 Enter your Fireworks API key: ··········


In [ ]:
llm = ChatFireworks(
    model="accounts/fireworks/models/llama-v3-70b-instruct",
    temperature=0.6,
    max_tokens=1024,
    stop=["\n\n"]
)

In [ ]:
prompt = [
    {"role":"system","content":"You are an expert data-scientist assistant."},
    {"role":"user","content":"Analyze the sentiment of this review:\n\n"
                           "\"The new movie was breathtaking, but a bit too long.\""}
]
resp = llm.invoke(prompt)
print("Sentiment Analysis →", resp.content)

Sentiment Analysis → A great review to analyze!


In [ ]:
template = """
You are a data-science assistant. Keep track of the convo:

{history}
User: {input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history","input"], template=template)
memory = ConversationBufferMemory(memory_key="history")

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

print(chain.run(input="Hey, what can you do?"))
print(chain.run(input="Analyze: 'The product arrived late, but support was helpful.'"))
print(chain.run(input="Based on that, would you recommend the service?"))

<ipython-input-10-28d37453855b>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history")
<ipython-input-10-28d37453855b>:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
<ipython-input-10-28d37453855b>:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run(input="Hey, what can you do?"))


I can assist with a wide range of data science tasks. I can help with data cleaning and preprocessing, exploratory data analysis, visualization, machine learning modeling, and more. I can also provide guidance on data science concepts and best practices.
I'd be happy to analyze the given text.
Based on the analysis, I would say that the recommendation is neutral or mixed. The customer mentions a negative aspect (the product arrived late), but also a positive aspect (the support was helpful). This suggests that the service has some room for improvement, but also has some redeeming qualities. Overall, it's not a strongly positive or negative review.


In [ ]:
class FetchURLTool(BaseTool):
    name: str = "fetch_url"
    description: str = "Fetch the main text (first 500 chars) from a webpage."

    def _run(self, url: str) -> str:
        resp = requests.get(url, timeout=10)
        doc = BeautifulSoup(resp.text, "html.parser")
        paras = [p.get_text() for p in doc.find_all("p")][:5]
        return "\n\n".join(paras)

    async def _arun(self, url: str) -> str:
        raise NotImplementedError

In [ ]:
class GenerateSQLTool(BaseTool):
    name: str = "generate_sql"
    description: str = "Generate a BigQuery SQL query (with comments) from a text description."

    def _run(self, text: str) -> str:
        prompt = f"""
-- Requirement:
-- {text}

-- Write a BigQuery SQL query (with comments) to satisfy the above.
"""
        return llm.invoke([{"role":"user","content":prompt}]).content

    async def _arun(self, text: str) -> str:
        raise NotImplementedError

tools = [FetchURLTool(), GenerateSQLTool()]

agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

result = agent.run(
    "Fetch https://en.wikipedia.org/wiki/ChatGPT "
    "and then generate a BigQuery SQL query that counts how many times "
    "the word 'model' appears in the page text."
)

print("\n🔍 Generated SQL:\n", result)



> Entering new AgentExecutor chain...
Thought: I need to fetch the webpage content first to get the text.

Action: fetch_url
Action Input: https://en.wikipedia.org/wiki/ChatGPT
Observation: 


ChatGPT is a generative artificial intelligence chatbot developed by the American company OpenAI and launched in 2022. It is based on large language models (LLMs) such as GPT-4o. ChatGPT can generate human-like conversational responses and enables users to refine and steer a conversation towards a desired length, format, style, level of detail, and language.[2] It is credited with accelerating the AI boom, which has led to ongoing rapid investment in and public attention to the field of artificial intelligence (AI).[3] Some observers have raised concern about the potential of ChatGPT and similar programs to displace human intelligence, enable plagiarism, or fuel misinformation.[4][5]


ChatGPT is built on OpenAI's proprietary series of generative pre-trained transformer (GPT) models and is fine